# Project 3: Subreddit Classification

## Problem Statement

Recently,there is an inflow of irrelevant posts on Apple and Android subreddits and Reddit has received many complaints from the end users. In an attempt to salvage their reputation, Reddit is looking to find a permanent fix to solve the issue by leveraging machine learning to distinguish between Apple and Android subreddit posts. Consequently, the load on the content moderators could be eased and millions of users could also benefit in finding relevant posts of their interest on the subreddits. 

The goal of this project is to develop a machine learning binary classifier and gain insights on the features which could accurately separate posts into Apple or Android subreddits, using natural language processing techniques. The performance of the different classifier models would be evaluated using the Receiver Operating Characteristic-Area Under Curve (ROC-AUC) metric which tells how good the classifier is in distinguishing between Apple and Android subreddits, with a success criteria of above 95% while preserving model generalizability.     

## Executive Summary






### Contents:
- [Data Collection](#Data-Collection)

## Data Collection

The Reddit API allows one to interact remotely to download posts from each subreddit, subject to a cap of 1000 posts. Since each typical request for posts only returns 25 entries, a custom function would need to be created to automate the process to collect sufficient data without overstressing the server such that a user could get blocked. This method requires a custom User-agent and a time.sleep() function to disguise the API call as coming from a Python programme.   

The goal is to gather as much data possible related to each subreddit post for the machine learning classifier to reasonably capture the relationships that may exist between input and output features.

In [1]:
import requests
import pandas as pd
import time
import random

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

%matplotlib inline

### Scrapping Apple Subreddit

In [2]:
posts = []
after = None

for a in range(40):
    url = 'https://www.reddit.com/r/apple.json'
    if after == None:
        current_url = url
    else:
        current_url = url + '?after=' + after
    print(current_url)
    
    # send request to url
    res = requests.get(current_url, headers={'User-agent': 'BBC Inc 1.0'})
    
    # check for errors
    if res.status_code != 200:
        print('Status error', res.status_code)
        break
    
    # get posts and add to [posts]
    current_dict = res.json()
    current_posts = [p['data'] for p in current_dict['data']['children']]
    posts.extend(current_posts)
    
    # get tag of last post on the page
    after = current_dict['data']['after']

    # generate a random sleep duration to look more 'natural'
    sleep_duration = random.randint(1,5)
    print(sleep_duration)
    time.sleep(sleep_duration)

https://www.reddit.com/r/apple.json
3
https://www.reddit.com/r/apple.json?after=t3_j06wzm
1
https://www.reddit.com/r/apple.json?after=t3_j02p2w
3
https://www.reddit.com/r/apple.json?after=t3_j05ws0
1
https://www.reddit.com/r/apple.json?after=t3_j0670r
3
https://www.reddit.com/r/apple.json?after=t3_iyu4vk
2
https://www.reddit.com/r/apple.json?after=t3_iybsyi
5
https://www.reddit.com/r/apple.json?after=t3_izcdcw
1
https://www.reddit.com/r/apple.json?after=t3_iysjo5
1
https://www.reddit.com/r/apple.json?after=t3_ix32rn
1
https://www.reddit.com/r/apple.json?after=t3_iwswjl
3
https://www.reddit.com/r/apple.json?after=t3_ivx31l
2
https://www.reddit.com/r/apple.json?after=t3_ivl1pj
4
https://www.reddit.com/r/apple.json?after=t3_iwa65i
1
https://www.reddit.com/r/apple.json?after=t3_ivxlf0
3
https://www.reddit.com/r/apple.json?after=t3_iuiasz
5
https://www.reddit.com/r/apple.json?after=t3_iuxujh
5
https://www.reddit.com/r/apple.json?after=t3_itphm8
2
https://www.reddit.com/r/apple.json?after=t3

In [3]:
len(posts)

985

In [4]:
# Check for unique posts
len(set([p['name'] for p in posts]))

808

In [5]:
df_apple = pd.DataFrame(posts)

In [6]:
# Drop duplicates 
df_apple.drop_duplicates(subset='name', keep="first", inplace=True)
df_apple.head()

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,thumbnail_height,top_awarded_type,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,ups,total_awards_received,media_embed,thumbnail_width,author_flair_template_id,is_original_content,user_reports,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,can_mod_post,score,approved_by,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,post_hint,content_categories,is_self,mod_note,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,view_count,archived,no_follow,is_crosspostable,pinned,over_18,preview,all_awardings,awarders,media_only,can_gild,spoiler,locked,author_flair_text,treatment_tags,visited,removed_by,num_reports,distinguished,subreddit_id,mod_reason_by,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,mod_reports,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,url_overridden_by_dest,link_flair_template_id
0,None,apple,\n\nWelcome to the daily Tech Support thread f...,t2_6l4z3,False,None,0,False,Daily Tech Support Thread - [September 27],"[{'e': 'text', 't': 'Official Megathread'}]",r/apple,False,6,megathread,0,NaN,None,False,t3_j0so5j,False,dark,0.68,None,public,7,0,{},NaN,None,False,[],None,False,False,None,{},Official Megathread,False,7,None,True,self,False,None,[],{},self,None,True,None,1.601248e+09,richtext,6,None,None,text,self.apple,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",None,new,None,None,False,True,False,False,False,{'images': [{'source': {'url': 'https://extern...,[],[],False,False,False,False,None,[],False,None,None,moderator,t5_2qh1f,None,None,,j0so5j,True,None,AutoModerator,None,66,False,all_ads,False,[],False,None,/r/apple/comments/j0so5j/daily_tech_support_th...,all_ads,True,https://www.reddit.com/r/apple/comments/j0so5j...,1800820,1.601219e+09,0,None,False,NaN,NaN
1,None,apple,"\n\nHello /r/Apple, and welcome to ""Shortcuts ...",t2_6l4z3,False,None,0,False,Shortcuts Sunday - [September 27],"[{'e': 'text', 't': 'Official Megathread'}]",r/apple,False,6,megathread,0,NaN,None,False,t3_j0que7,False,dark,0.81,None,public,20,0,{},NaN,None,False,[],None,False,False,None,{},Official Megathread,False,20,None,True,self,False,None,[],{},self,None,True,None,1.601241e+09,richtext,6,None,None,text,self.apple,True,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",None,None,None,None,False,True,False,False,False,{'images': [{'source': {'url': 'https://extern...,[],[],False,False,False,False,None,[],False,None,None,moderator,t5_2qh1f,None,None,,j0que7,True,None,AutoModerator,None,5,False,all_ads,False,[],False,None,/r/apple/comments/j0que7/shortcuts_sunday_sept...,all_ads,True,https://www.reddit.com/r/apple/comments/j0que7...,1800820,1.601212e+09,0,None,False,NaN,NaN
2,None,apple,,t2_78w5nxue,False,None,0,False,iOS 14: 'Phoenix 2' Space Shooter Delivers Pla...,"[{'e': 'text', 't': 'iOS'}]",r/apple,False,6,ios,0,83.0,None,False,t3_j0rz3p,False,dark,0.97,None,public,1660,3,{},140.0,None,False,[],None,False,False,None,{},iOS,False,1660,None,False,https://b.thumbs.redditmedia.com/i8UjN90iT4042...,False,None,[],{},link,None,False,None,1.601246e+09,richtext,6,None,None,text,macrumors.com,True,None,None,None,None,None,False,False,False,False,False,{'images': [{'source': {'url': 'https://extern...,"[{'giver_coin_reward': None, 'subreddit_id': N...",[],False,False,False,False,None,[],False,None,None,None,t5_2qh1f,None,None,,j0rz3p,True,None,redditer_09,None,92,True,all_ads,False,[],False,None,/r/apple/

In [2]:
col_keep =  ['subreddit', 'selftext', 'title','upvote_ratio','score','num_comments','author' ]

In [8]:
df_apple = df_apple[col_keep]

In [9]:
df_apple.head()

,subreddit,selftext,title,upvote_ratio,score,num_comments,author
0,apple,\n\nWelcome to the daily Tech Support thread f...,Daily Tech Support Thread - [September 27],0.68,7,66,AutoModerator
1,apple,"\n\nHello /r/Apple, and welcome to ""Shortcuts ...",Shortcuts Sunday - [September 27],0.81,20,5,AutoModerator
2,apple,,iOS 14: 'Phoenix 2' Space Shooter Delivers Pla...,0.97,1660,92,redditer_09
3,apple,,iOS 14: How to stop your AirPods automatically...,0.95,233,44,PJ09
4,apple,,There was no NASA Astronomy Picture of the Day...,0.97,4167,158,TrailWhale


In [10]:
# Export to csv
df_apple.to_csv('../data/apple_subreddit v2.csv', index = False)

### Scrapping Android Subreddit

In [2]:
posts = []
after = None

for a in range(40):
    url = 'https://www.reddit.com/r/Android.json'
    if after == None:
        current_url = url
    else:
        current_url = url + '?after=' + after
    print(current_url)
    
    # send request to url
    res = requests.get(current_url, headers={'User-agent': 'BBC Inc 1.0'})
    
    # check for errors
    if res.status_code != 200:
        print('Status error', res.status_code)
        break
    
    # get posts and add to [posts]
    current_dict = res.json()
    current_posts = [p['data'] for p in current_dict['data']['children']]
    posts.extend(current_posts)
    
    # get tag of last post on the page
    after = current_dict['data']['after']

    # generate a random sleep duration to look more 'natural'
    sleep_duration = random.randint(1,5)
    print(sleep_duration)
    time.sleep(sleep_duration)

https://www.reddit.com/r/Android.json
5
https://www.reddit.com/r/Android.json?after=t3_j04ato
1
https://www.reddit.com/r/Android.json?after=t3_iyysc1
2
https://www.reddit.com/r/Android.json?after=t3_ixszud
3
https://www.reddit.com/r/Android.json?after=t3_iy9bir
5
https://www.reddit.com/r/Android.json?after=t3_ivrc4d
2
https://www.reddit.com/r/Android.json?after=t3_ivm5uq
3
https://www.reddit.com/r/Android.json?after=t3_iuffvg
1
https://www.reddit.com/r/Android.json?after=t3_itfsx0
5
https://www.reddit.com/r/Android.json?after=t3_isv51c
1
https://www.reddit.com/r/Android.json?after=t3_irwf64
1
https://www.reddit.com/r/Android.json?after=t3_iqoyfi
5
https://www.reddit.com/r/Android.json?after=t3_iphepi
3
https://www.reddit.com/r/Android.json?after=t3_ioxv11
4
https://www.reddit.com/r/Android.json?after=t3_io6oae
3
https://www.reddit.com/r/Android.json?after=t3_inkb4u
4
https://www.reddit.com/r/Android.json?after=t3_imfsj4
1
https://www.reddit.com/r/Android.json?after=t3_il5i6u
5
https://

In [3]:
len(posts)

977

In [4]:
# Check for unique posts
len(set([p['name'] for p in posts]))

726

In [5]:
df_android = pd.DataFrame(posts)

In [6]:
# Drop duplicates 
df_android.drop_duplicates(subset='name', keep="first", inplace=True)
df_android.head()

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,thumbnail_height,top_awarded_type,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,ups,total_awards_received,media_embed,thumbnail_width,author_flair_template_id,is_original_content,user_reports,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,can_mod_post,score,approved_by,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,content_categories,is_self,mod_note,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,view_count,archived,no_follow,is_crosspostable,pinned,over_18,all_awardings,awarders,media_only,can_gild,spoiler,locked,author_flair_text,treatment_tags,visited,removed_by,num_reports,distinguished,subreddit_id,mod_reason_by,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,mod_reports,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,post_hint,url_overridden_by_dest,preview,link_flair_template_id,author_cakeday
0,None,Android,"Note 1. Join us at /r/MoronicMondayAndroid, a ...",t2_6l4z3,False,None,0,False,Moronic Monday (Sep 28 2020) - Your weekly que...,[],r/Android,False,6,None,0,NaN,None,False,t3_j1a3yp,False,dark,0.82,,public,21,0,{},NaN,None,False,[],None,False,False,None,{},None,False,21,None,True,self,False,robot,[],{},None,True,None,1.601321e+09,text,6,None,None,text,self.Android,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",None,new,None,None,False,False,False,False,False,[],[],False,False,False,False,*beep boop*,[],False,None,None,moderator,t5_2qlqh,None,None,,j1a3yp,True,None,AutoModerator,None,45,False,all_ads,False,[],False,dark,/r/Android/comments/j1a3yp/moronic_monday_sep_...,all_ads,True,https://www.reddit.com/r/Android/comments/j1a3...,2268461,1.601292e+09,0,None,False,NaN,NaN,NaN,NaN,NaN
1,None,Android,,t2_65wds,False,None,0,False,The Home Depot is selling Google’s new Chromec...,[],r/Android,False,6,None,0,73.0,None,False,t3_j1e68n,False,dark,0.97,None,public,3317,0,{},140.0,None,False,[],None,False,False,None,{},None,False,3317,None,False,https://b.thumbs.redditmedia.com/scixK6uf0GnqF...,False,None,[],{},None,False,None,1.601336e+09,text,6,None,None,text,theverge.com,True,None,None,None,None,None,False,False,False,False,False,[],[],False,False,False,False,None,[],False,None,None,None,t5_2qlqh,None,None,,j1e68n,True,None,zbhoy,None,469,True,all_ads,False,[],False,None,/r/Android/comments/j1e68n/the_home_depot_is_s...,all_ads,False,https://www.theverge.com/2020/9/28/21459849/go...,2268461,1.601307e+09,4,None,False,link,https://www.theverge.com/2020/9/28/21459849/go...,{'images': [{'source': {'url': 'https://extern...,NaN,NaN
2,None,Android,,t2_31mkizvx,False,None,0,False,Developer boots Android 11 on 22 older devices...,[],r/Android,False,6,None,0,93.0,None,False,t3_j1gjze,False,dark,0.95,None,public,138,0,{},140.0,None,False,[],None,False,False,None,{},None,False,138,None,False,https://a.thumbs.redditmedia.com/nDtW5EcfvEXqa...,False,None,[],{},None,False,None,1.601343e+09,text,6,None,None,text,xda-developers.com,False,None,None,None,None,None,False,False,False,False,False,[],[],False,False,False,False,None,[],False,None,None,None,t5_2qlqh,None,None,,j1gjze,True,None,apmcruZ,None,23,True,all_ads,False,[],False,None,/r/Android/comments/j1gjze/developer_boots_and...,all_ads,False,https://www.xda-developers.com/developer-boots...,2268461,1.601314e+09,0,None,False,link,https://www.xda-developers.com/developer-boots...,{'images': [{'source': {'url': 'https://extern...,NaN,NaN
3,No

In [9]:
df_android = df_android[col_keep]
df_android.head()

,subreddit,selftext,title,upvote_ratio,score,num_comments,author
0,Android,"Note 1. Join us at /r/MoronicMondayAndroid, a ...",Moronic Monday (Sep 28 2020) - Your weekly que...,0.82,21,45,AutoModerator
1,Android,,The Home Depot is selling Google’s new Chromec...,0.97,3317,469,zbhoy
2,Android,,Developer boots Android 11 on 22 older devices...,0.95,138,23,apmcruZ
3,Android,,[MKBHD] The Galaxy S20 Fan Edition: Hear Me Out!,0.75,38,42,HayashiSawaryo
4,Android,Recently a member of my family got Samsung Not...,Realised how annoying updating is without A/B ...,0.88,131,86,whamenrespecter69


In [27]:
df_android.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 731 entries, 0 to 730
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   subreddit     731 non-null    object 
 1   selftext      731 non-null    object 
 2   title         731 non-null    object 
 3   upvote_ratio  731 non-null    float64
 4   score         731 non-null    int64  
 5   num_comments  731 non-null    int64  
 6   author        731 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 45.7+ KB


In [10]:
# Export to csv
df_android.to_csv('../data/android_subreddit v3.csv', index = False)

### Merging data collection

In [11]:
df_android_v1 = pd.read_csv('../data/android_subreddit.csv')
df_apple_v1 = pd.read_csv('../data/apple_subreddit.csv')
df_android_v2 = pd.read_csv('../data/android_subreddit v2.csv')
df_apple_v2 = pd.read_csv('../data/apple_subreddit v2.csv')
df_android_v3 = pd.read_csv('../data/android_subreddit v3.csv')

In [12]:
# merge dataframes
combined_android_df = df_android_v2.append(df_android_v1, ignore_index=True)
combined_apple_df = df_apple_v2.append(df_apple_v1, ignore_index=True)

In [13]:
combined_android_df = combined_android_df.append(df_android_v3, ignore_index=True)

In [14]:
# Drop duplicates 
combined_android_df.drop_duplicates(subset='title', keep="first", inplace=True)
combined_android_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 776 entries, 0 to 1476
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   subreddit     776 non-null    object 
 1   selftext      200 non-null    object 
 2   title         776 non-null    object 
 3   upvote_ratio  776 non-null    float64
 4   score         776 non-null    int64  
 5   num_comments  776 non-null    int64  
 6   author        776 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 48.5+ KB


In [15]:
combined_apple_df.drop_duplicates(subset='title', keep="first", inplace=True)
combined_apple_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 937 entries, 0 to 1607
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   subreddit     937 non-null    object 
 1   selftext      225 non-null    object 
 2   title         937 non-null    object 
 3   upvote_ratio  937 non-null    float64
 4   score         937 non-null    int64  
 5   num_comments  937 non-null    int64  
 6   author        937 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 58.6+ KB


In [16]:
# Export to csv
combined_android_df.to_csv('../data/android_subreddit_final.csv', index = False)
combined_apple_df.to_csv('../data/apple_subreddit_final.csv', index = False)

### Data Dictionary

|Feature|Type|Dataset|Description|
|---|---|---|---|
|subreddit|str|android/ apple subreddits|The name of the subreddit|
|selftext|str|android/ apple subreddits|The body text of each subreddit post| 
|title|str|android/ apple subreddits|The title of each subreddit post|
|upvote_ratio|float|android/ apple subreddits|The number of upvotes a post has, divided by the total number of votes the post received|
|score|float|android/ apple subreddits|The number of upvotes a post has|
|num_comments|float|android/ apple subreddits|The number of upvotes a post has| 